NameError: name 'train' is not defined

In [83]:
%store -r 
#restore train, validate, and test dataframes from other notebook
train.head()

,file_name,hair_color,eyeglasses,smiling,young,human
2308,2531,2,1,1,1,-1
82,92,1,1,1,1,-1
4401,4830,-1,-1,-1,1,1
817,907,0,-1,-1,1,1
471,522,1,-1,1,1,1


In [ ]:
train_human = train[['file_name', 'human_-1', 'human_1']].reset_index(drop=True)
validate_human = validate[['file_name', 'human_-1', 'human_1']].reset_index(drop=True)
test_human = test[['file_name', 'human_-1', 'human_1']].reset_index(drop=True)

In [84]:
train_human = train[['file_name', 'human']]
train_human = train_human.replace(to_replace=1, value=1).replace(to_replace=-1, value=0).reset_index(drop=True)
validate_human = validate[['file_name', 'human']]
validate_human = validate_human.replace(to_replace=1, value=1).replace(to_replace=-1, value=0).reset_index(drop=True)
test_human = test[['file_name', 'human']]
test_human = test_human.replace(to_replace=1, value=1).replace(to_replace=-1, value=0).reset_index(drop=True)

In [85]:

n_human = train_human['human'].value_counts()

print(n_human)

0    1535
1    1204
Name: human, dtype: int64


In [86]:
from keras.utils import to_categorical
print(to_categorical(train_human['human']))

[[1. 0.]
 [1. 0.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [ ]:
train_human.head()

In [87]:
validate_human.head(20)

,file_name,human
0,4535,1
1,3316,1
2,652,1
3,835,1
4,2302,1
5,4509,0
6,3011,1
7,4343,1
8,853,0
9,2282,0


In [ ]:
train_human.head()

In [93]:
#https://jovianlin.io/cat-crossentropy-vs-sparse-cat-crossentropy/

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.layers.advanced_activations import LeakyReLU

K.set_image_dim_ordering('th')

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(3, 128, 128)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
#model.add(LeakyReLU(alpha=0.3))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [94]:
import os
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

batch_size = 32

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0,
        zoom_range=0,
        horizontal_flip=False,
        data_format='channels_first')

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255, data_format='channels_first')

#Takes the dataframe and the path to a directory and 
#generates batches of augmented/normalized data.

trainPath = './images/train/'
validatePath = './images/validate/'
#labels_cols = list(train)[1:]
labels_cols = list(train_human)[1:]

train_gen = train_datagen.flow_from_dataframe(
            dataframe = train_human,
            directory = trainPath,
            x_col = "file_name",
            y_col = 'human',
            has_ext = False,
            target_size=(128,128),
            color_mode='rgb',
            class_mode='binary',
            batch_size=batch_size,
            shuffle=True,
            seed=123)

validation_gen = test_datagen.flow_from_dataframe(
            dataframe = validate_human,
            directory = validatePath,
            x_col = "file_name",
            y_col = 'human',
            has_ext = False,
            target_size=(128,128),
            color_mode='rgb',
            class_mode='binary',
            batch_size=batch_size,
            shuffle=True,
            seed=123)

Found 2739 images belonging to 2 classes.
Found 913 images belonging to 2 classes.


In [2]:
import os
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

batch_size = 32

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0,
        zoom_range=0,
        horizontal_flip=False,
        data_format='channels_first')

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255, data_format='channels_first')

#Takes the dataframe and the path to a directory and 
#generates batches of augmented/normalized data.

trainPath = './images/train/'
validatePath = './images/validate/'

train_gen = train_datagen.flow_from_directory(
            directory = trainPath,
            target_size=(128,128),
            class_mode='binary',
            batch_size=batch_size,
            shuffle=True,
            seed=123)

validation_gen = test_datagen.flow_from_directory(
            directory = validatePath,
            target_size=(128,128),
            class_mode='binary',
            batch_size=batch_size,
            shuffle=True,
            seed=123)

Found 2739 images belonging to 2 classes.
Found 913 images belonging to 2 classes.


In [ ]:
import matplotlib.pyplot as plt
import skimage
%matplotlib inline

def img_info(ndarray, gray=0):
    '''displays image and shape; gray is a flag used by matplotlib'''
    if gray:
        plt.imshow(ndarray,cmap='gray')
    else:
        plt.imshow(ndarray);
    print('Image shape:', ndarray.shape)
    
def image_data_from_number(imageNumber, imagesPath):
    filename = imagesPath+str(imageNumber)+'.png'
    image = skimage.io.imread(filename, as_gray=False)
    return image#raw image data

img = image_data_from_number(2015, validatePath)
img_info(img)

In [95]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from sklearn.utils import class_weight

#class_weights = class_weight.compute_class_weight('balanced',
                                                 #['cartoon', 'human'],
                                                 #train_human.head(2720)['human'])

earlyStopping = EarlyStopping(monitor='val_loss',
                              min_delta=0.001,
                              patience=0,
                              verbose=1, 
                              mode='auto',
                              restore_best_weights=True)

LR = ReduceLROnPlateau(monitor='val_loss', 
                       factor=0.2, 
                       patience=0, 
                       verbose=1, 
                       mode='auto', 
                       min_delta=0.01, 
                       cooldown=0, 
                       min_lr=0.001)
train_gen.reset()
validation_gen.reset()

STEP_SIZE_TRAIN=train_gen.n/train_gen.batch_size
STEP_SIZE_VALID=validation_gen.n/validation_gen.batch_size

In [96]:
print(STEP_SIZE_TRAIN, STEP_SIZE_VALID)
print(STEP_SIZE_TRAIN*train_gen.batch_size, STEP_SIZE_VALID*validation_gen.batch_size)
model.summary()

85.59375 28.53125
2739.0 913.0
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 32, 128, 128)      896       
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 32, 126, 126)      9248      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 32, 63, 63)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 32, 63, 63)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 32, 63, 63)        9248      
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 64, 61, 61)        18496     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 64, 30, 3

In [97]:
model.fit_generator(generator=train_gen,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_gen,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=50,
                    callbacks=[earlyStopping, LR],
                    class_weight = 'auto')

#model.save_weights('.h5')
model.save_weights('human_detection_from_dataframe.h5')

Epoch 1/50
86/85 [==============================] - 346s 4s/step - loss: 0.3317 - acc: 0.8319 - val_loss: 4.7246 - val_acc: 0.0131
Epoch 2/50
86/85 [==============================] - 368s 4s/step - loss: 0.0257 - acc: 0.9902 - val_loss: 13.9587 - val_acc: 0.0000e+00
Restoring model weights from the end of the best epoch

Epoch 00002: ReduceLROnPlateau reducing learning rate to 0.001.
Epoch 00002: early stopping


In [100]:
validation_gen = test_datagen.flow_from_dataframe(
            dataframe = validate_human,
            directory = validatePath,
            x_col = "file_name",
            y_col = 'human',
            has_ext = False,
            target_size=(128,128),
            color_mode='rgb',
            class_mode='binary',
            batch_size=1,
            shuffle=False)

validation_gen.reset()

model.evaluate_generator(generator=validation_gen, steps=913, verbose=1)

Found 913 images belonging to 2 classes.
913/913 [==============================] - 47s 51ms/step


[4.724596299039789, 0.013143483023001095]

In [1]:
validation_gen = test_datagen.flow_from_directory(
            directory = validatePath,
            target_size=(128,128),
            class_mode='binary',
            batch_size=1,
            shuffle=False)

validation_gen.reset()

model.evaluate_generator(generator=validation_gen, steps=913, verbose=1)

NameError: name 'test_datagen' is not defined

In [65]:
testPath = './images/test/'
test_gen = test_datagen.flow_from_directory(
            directory = testPath,
            target_size=(128,128),
            class_mode='binary',
            batch_size=1,
            shuffle=False)

test_gen.reset()

model.evaluate_generator(generator=test_gen, steps=913, verbose=1)

Found 913 images belonging to 2 classes.
913/913 [==============================] - 46s 50ms/step


[0.00939483462716854, 0.9967141292442497]

In [66]:
test_gen.reset()
probabilities = model.predict_generator(test_gen,verbose=1,steps=len(test_gen))

913/913 [==============================] - 45s 49ms/step


In [63]:
def get_true_y(generator, steps):
    generator.reset()
    actual_values = []
    i=0
    for batch_x, batch_y in generator:
        i+=1
        print("iteration count: ", i, "true_y: ", batch_y)
        actual_values.append(batch_y)
        #print(batch_x)
        if i == steps:
            break
    
    return actual_values

true_y = get_true_y(test_gen, 913)

iteration count:  1 true_y:  [0.]
iteration count:  2 true_y:  [0.]
iteration count:  3 true_y:  [1.]
iteration count:  4 true_y:  [1.]
iteration count:  5 true_y:  [1.]
iteration count:  6 true_y:  [0.]
iteration count:  7 true_y:  [0.]
iteration count:  8 true_y:  [0.]
iteration count:  9 true_y:  [0.]
iteration count:  10 true_y:  [0.]
iteration count:  11 true_y:  [0.]
iteration count:  12 true_y:  [0.]
iteration count:  13 true_y:  [0.]
iteration count:  14 true_y:  [0.]
iteration count:  15 true_y:  [0.]
iteration count:  16 true_y:  [1.]
iteration count:  17 true_y:  [0.]
iteration count:  18 true_y:  [0.]
iteration count:  19 true_y:  [1.]
iteration count:  20 true_y:  [1.]
iteration count:  21 true_y:  [0.]
iteration count:  22 true_y:  [0.]
iteration count:  23 true_y:  [0.]
iteration count:  24 true_y:  [0.]
iteration count:  25 true_y:  [0.]
iteration count:  26 true_y:  [0.]
iteration count:  27 true_y:  [0.]
iteration count:  28 true_y:  [0.]
iteration count:  29 true_y: 

iteration count:  268 true_y:  [1.]
iteration count:  269 true_y:  [0.]
iteration count:  270 true_y:  [0.]
iteration count:  271 true_y:  [1.]
iteration count:  272 true_y:  [1.]
iteration count:  273 true_y:  [0.]
iteration count:  274 true_y:  [0.]
iteration count:  275 true_y:  [0.]
iteration count:  276 true_y:  [1.]
iteration count:  277 true_y:  [0.]
iteration count:  278 true_y:  [0.]
iteration count:  279 true_y:  [0.]
iteration count:  280 true_y:  [0.]
iteration count:  281 true_y:  [1.]
iteration count:  282 true_y:  [1.]
iteration count:  283 true_y:  [0.]
iteration count:  284 true_y:  [1.]
iteration count:  285 true_y:  [1.]
iteration count:  286 true_y:  [1.]
iteration count:  287 true_y:  [0.]
iteration count:  288 true_y:  [1.]
iteration count:  289 true_y:  [1.]
iteration count:  290 true_y:  [1.]
iteration count:  291 true_y:  [1.]
iteration count:  292 true_y:  [1.]
iteration count:  293 true_y:  [1.]
iteration count:  294 true_y:  [1.]
iteration count:  295 true_y

iteration count:  519 true_y:  [0.]
iteration count:  520 true_y:  [0.]
iteration count:  521 true_y:  [0.]
iteration count:  522 true_y:  [0.]
iteration count:  523 true_y:  [0.]
iteration count:  524 true_y:  [0.]
iteration count:  525 true_y:  [0.]
iteration count:  526 true_y:  [1.]
iteration count:  527 true_y:  [1.]
iteration count:  528 true_y:  [0.]
iteration count:  529 true_y:  [1.]
iteration count:  530 true_y:  [1.]
iteration count:  531 true_y:  [1.]
iteration count:  532 true_y:  [1.]
iteration count:  533 true_y:  [1.]
iteration count:  534 true_y:  [0.]
iteration count:  535 true_y:  [1.]
iteration count:  536 true_y:  [1.]
iteration count:  537 true_y:  [0.]
iteration count:  538 true_y:  [0.]
iteration count:  539 true_y:  [1.]
iteration count:  540 true_y:  [1.]
iteration count:  541 true_y:  [1.]
iteration count:  542 true_y:  [0.]
iteration count:  543 true_y:  [1.]
iteration count:  544 true_y:  [0.]
iteration count:  545 true_y:  [0.]
iteration count:  546 true_y

iteration count:  758 true_y:  [1.]
iteration count:  759 true_y:  [1.]
iteration count:  760 true_y:  [1.]
iteration count:  761 true_y:  [0.]
iteration count:  762 true_y:  [0.]
iteration count:  763 true_y:  [0.]
iteration count:  764 true_y:  [0.]
iteration count:  765 true_y:  [0.]
iteration count:  766 true_y:  [1.]
iteration count:  767 true_y:  [0.]
iteration count:  768 true_y:  [0.]
iteration count:  769 true_y:  [0.]
iteration count:  770 true_y:  [1.]
iteration count:  771 true_y:  [0.]
iteration count:  772 true_y:  [0.]
iteration count:  773 true_y:  [1.]
iteration count:  774 true_y:  [0.]
iteration count:  775 true_y:  [0.]
iteration count:  776 true_y:  [1.]
iteration count:  777 true_y:  [0.]
iteration count:  778 true_y:  [0.]
iteration count:  779 true_y:  [1.]
iteration count:  780 true_y:  [1.]
iteration count:  781 true_y:  [0.]
iteration count:  782 true_y:  [0.]
iteration count:  783 true_y:  [1.]
iteration count:  784 true_y:  [1.]
iteration count:  785 true_y

In [74]:
print(np.rint(probabilities).flatten())

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [72]:
np.rint(probabilities)

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],

In [79]:
import pandas as pd
output_df = pd.DataFrame(
    {'filename': test_gen.filenames,
     'y_true': test_gen.classes,
     'y_pred': list(map(int, np.rint(probabilities).flatten()))
    })

In [81]:
output_df_wrong = output_df.query('y_true != y_pred')
output_df_wrong

,filename,y_true,y_pred
34,cartoon/1326.png,0,1
231,cartoon/3036.png,0,1
478,cartoon/661.png,0,1


In [78]:
from sklearn.metrics import confusion_matrix
y_true = test_gen.classes
y_pred = np.rint(probabilities)
labels = ['cartoon','human']
confusion_matrix(y_true=y_true, y_pred=y_pred)

array([[515,   3],
       [  0, 395]])

In [ ]:
from six import next

testPath = './images/test/'

test_gen = test_datagen.flow_from_dataframe(
        dataframe = test_human,
        directory = testPath,
        x_col = "file_name",
        y_col = ['human_-1','human_1'],
        has_ext = False,
        target_size=(150,150),
        color_mode='rgb',
        class_mode='other',
        batch_size=1,
        shuffle=False,
        seed=123)

def generator_with_true_classes(model, generator):
    while True:
        x, y = next(generator)
        yield x, model.predict(x), y


nb_of_samples = 0
nb_of_samples_to_compute = 100 # set your own value
for x, y_pred, y_true in generator_with_true_classes(model, test_gen):
    # do something with data, eg. print it.
    nb_of_samples += 1
    if nb_of_samples == nb_of_samples_to_compute:
         break

In [ ]:
import numpy as np
validate_human.head()
img = image_data_from_number(1110, validatePath)
img_info(img)
print(np.argmax(y_pred, axis=1), y_true)

In [ ]:
import pprint

testPath = './images/test/'

test_gen = test_datagen.flow_from_dataframe(
        dataframe = test_human,
        directory = testPath,
        x_col = "file_name",
        y_col = labels_cols,
        has_ext = False,
        target_size=(150,150),
        color_mode='rgb',
        class_mode='other',
        batch_size=1,
        shuffle=False)

test_gen.reset()

probabilities = model.predict_generator(test_gen,verbose=1,steps=len(test_gen))

In [ ]:
pprint.pprint(probabilities)

In [ ]:
test_human.head()

In [ ]:
print(probabilities)

In [14]:
import numpy as np
print(np.argmax(probabilities, axis=1))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [ ]:
test_human.head()